##### 一、安装所使用的库
1. tensorflow
2. opencv-python
3. numpy

In [2]:
!pip install tensorflow
!pip install numpy
!pip install opencv-python

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 11.2 MB/s eta 0:00:0000:0100:01


##### 二、导入所使用的库

In [3]:
import tensorflow as tf
import numpy as np
import cv2

2024-02-14 20:53:06.351475: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-14 20:53:06.403847: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-14 20:53:06.404621: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-14 20:53:07.862199: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


##### 三、读取tflite文件并获取输入、输出参数
1. 使用tf.lite.Interpreter(model_path=)函数读取tflite模型
2. 根据1.所返回的对象调用get_input_details()/get_output_details()函数获取输入、输出参数

In [6]:
tflite_path = "../Flower_Reco/app/src/main/assets/graph.tflite"

interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

}##### 四、读取测试图片并调整大小
首先需要打印input_details获取输入图片的维度
然后再使用cv2库方法imread读取、resize修改大小
最后更改数据类型，进行归一化并增加维度

In [7]:
print(input_details)

[{'name': 'input', 'index': 89, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([  1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [19]:
imgs_path = "../dataSets/flower_photos/"

# 以牵牛花/7.jpg为例
sample = imgs_path + "牵牛花/7.jpg"
img = cv2.imread(sample)
img = cv2.resize(img,(224,224)).astype(np.float32)
img = img / 255.0
img = np.expand_dims(img, axis=0)

##### 五、调用模型并获得结果

获得结果使用np.where(值=np.max)

In [20]:
interpreter.set_tensor(input_details[0]['index'],img)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
output_data = np.squeeze(output_data)
print(output_data)

[4.2159849e-14 4.0751988e-09 1.4678649e-13 3.6809187e-13 9.9999976e-01
 9.3002255e-09 2.2809490e-07 3.0968401e-15 1.1022881e-13 3.9573810e-13]


In [21]:
# 输出结果是长度为10（对应0-9）的一维数据，最大值的下标就是预测的数字
print('result:{}'.format((np.where(output_data == np.max(output_data)))[0][0]))

result:4
